# Тестирование

Хорошая привычка - писать код и тесты к нему.  Если у вас уже есть тесты, то когда вы будете редактировать код и добавлять новую функциональность, вы сможете легко проверить, что вы не сломали что-то, что уже работало хорошо.

Если код большой, то каждый тест должен проверять его небольшую логическую часть. 

## unittest

Вспомним, как мы писали класс преподавателя и доставали информацию о преподавателях со страницы hse.ru/org/persons.  Предположим, у нашего преподавателя есть атрибуты *name* "имя", *surname* "фамилия", *patr* "отчество". На странице полное имя преподавателя записано как текст внутри одной ссылки, так что мы можем его легко достать, например "Азизова Роза Мухамаджановна". Представим себе, что мы написали функцию *parse_name* , которая получает на вход такую строку и делит ее на имя, фамилию и отчество.

In [4]:
def parse_name(name):
    parts = name.strip().split()
    surname, name, patr = '', '', ''
    if len(parts) == 1:
        name = parts[0]
    elif len(parts) == 2:
        surname, name = parts[0], parts[1]
    elif len(parts) == 3:
        surname, name, patr = parts
    elif len(parts) > 3:
        surname, name, patr = parts[0], ' '.join(parts[1:-1]), parts[-1]
    return surname, name, patr

('Азизова', 'Роза', 'Мухамаджановна')
('', 'Вася', '')
('Айано', 'Таканори', '')
('Азизов', 'Анатолий Эмир', 'Алиевич')


А теперь напишем код, который эту функцию тестирует. Тесты в `unittest` организуются в группы внутри класса `TestCase`. 

Для каждой логической части функции нужно написать отдельный тест, который эту часть проверяет. Эти тесты оформляются в виде методов класса, названия методов начинаются со слова `test`. 

In [ ]:
import unittest

class ParseNameTestCase(unittest.TestCase):
    def test_one_word(self):
        self.assertEqual(('', 'Имя', ''), parse_name('Имя'))
    def test_two_words(self):
        self.assertEqual(('Фамилия', 'Имя', ''), parse_name('Фамилия Имя'))
    def test_three_words(self):
        self.assertEqual(('Фамилия', 'Имя', 'Отчество'), parse_name('Фамилия Имя Отчество'))
    def test_more_words(self):
        self.assertEqual(('Фамилия', 'Имя Имя', 'Отчество'), parse_name('Фамилия Имя Имя Отчество'))
    def test_no_words(self):
        self.assertEqual(('', '', ''), parse_name(''))
        
if __name__ == "__main__":
    unittest.main()

Работает:

In [6]:
for i in ['Азизова Роза Мухамаджановна', 'Вася', 'Айано Таканори', 'Азизов Анатолий Эмир Алиевич']:
    print(parse_name(i))

('Азизова', 'Роза', 'Мухамаджановна')
('', 'Вася', '')
('Айано', 'Таканори', '')
('Азизов', 'Анатолий Эмир', 'Алиевич')


Если при выполнении тестов не возникло ошибок, то ура - мы успешно протестировали наш код. 

В классе `TestCase` есть вспомогательные методы для проверки разных тестов: 
* `assertEqual` - для проверки равенства результата функции и заданного ответа,
* `assertTrue` - для проверки булевых выражений, 
* `assertRaises` - чтобы проверить, что в нужных местах вызываются нужные исключения.

Можно писать свои вспомогательные методы, главное, чтобы их называния не начинались с `test`.

Например, есть еще два полезных вспомогательных метода, которые используются, чтобы настроить среду тестирования:
* `setUp`
* `tearDown` 

Эти методы вызываются соответственно до и после каждого теста и нужны для того, чтобы убедиться, что каждый тест работает изолированно и на него не влияет, например, результат работы прошлого теста.  В качестве примера - вот такой `TestCase`, который создает временную папку перед выполнением каждого теста и удаляет ее после завершения каждого теста:

In [ ]:
class MyTest(TestCase):
    def setUp(self):
    self.test_dir = TemporaryDirectory()
    def tearDown(self):
    self.test_dir.cleanup()
    # а тут сами тесты
    # …

Обычно пишут один `TestCase` для набора связанных тестов. Иногда разумно написать один `TestCase` для одной функции, у которой много вариантов пограничных случаев. Иногда разумно написать один `TestCase` на все функции внутри одного модуля. Хорошо написать `TestCase` для тестирования отдельного класса и его методов. 

Еще пример - числа Фибоначчи:

In [ ]:
def fib(n):
    if type(n) != int:
        return -1
    a,b = 1,1
    for i in range(n-1):
        a,b = b,a+b
    return a


class FibTest(unittest.TestCase):
    def test_math(self):
        self.assertEqual(fib(0), 1)
        self.assertEqual(fib(1), 1)
        self.assertEqual(fib(8), 21)

    def test_bad_data(self):
        self.assertEqual(fib('вауу'), -1)

## doctest

Модуль `doctest` находит кусочки кода, которые выглядят как интерактивные сессии питона, выполняет эти сессии и проверяет, что в коде все выглядит так же, как в реальной сессии. Доктесты обычно используют в документации к коду, чтобы быстро продемонстрировать как он работает. 

In [ ]:
def find_position(s, text):
    """
    Находит позицию строки s в строке text.
    :param s: str - заданная подстрока
    :param text: str - строка, в которой выполняется поиск
    :return: int - позиция подстроки в строке text или -1, если подстрока не найдена

    >>> find_position('1', '331')
    2
    >>> find_position('aa', 'abcd')
    -1
    >>> find_position('abcd', 'fabcdffff')
    1
    """
    return text.index(s)

if __name__ == '__main__':
    import doctest
    doctest.testmod()

# Документация

Чтобы вашим кодом могли пользоваться другие люди (или даже вы сами через год), нужно куда-нибудь записать, что в нем происходит. В питоне можно прикреплять документацию прямо к соответствующему кусочку кода (классу, функции или модулю). Можно добавить документацию с помощью *docstring* после строчки с `def` (см. пример выше или ниже).

In [7]:
def palindrome(word):
    """Return True if the given word is a palindrome."""
    return word == word[::-1]

*docstring* можно достать через атрибут `__doc__`.

In [8]:
print(palindrome.__doc__)

Return True if the given word is a palindrome.


Существует руководство по написанию *docstring*: https://www.python.org/dev/peps/pep-0257/

Докстринги нужно писать к каждому классу:
* Первая строчка  -- одно предложение, описывающее смысл этого класса. Например, про класс преподавателя можно было написать "Represents a professor."
* Затем нужно описать важные детали для работы с этим классом: методы или важные атрибуты.

Нужно писать докстринги к каждой функции или методу класса:
* Первая строчка - краткое описание того, что делает функция.
* Затем может быть более подробное описание.
* Если функция принимает какие-то аргументы, нужно объяснить их значения.
* Если функция что-то возвращает, нужно описать возвращаемое значение.
* Если функция вызывает исключение, нужно это указать.

Вот еще пример:

In [ ]:
def find_anagrams(word, dictionary):
    """Find all anagrams for a word.
    
    This function only runs as fast as the test for membership in the 'dictionary' container. 
    It will be slow if the dictionary is a list and fast if it’s a set.
    
    Args:
        word: String of the target word.
        dictionary: Container with all strings that are known to be actual words.
      
    Returns:
        List of anagrams that were found. Empty if none were found.
    """

# Задание 

## `kwiq`

1) Напишите, функцию `kwiq`, которая принимает на вход слово и текст, а возвращает сниппеты с этим словом из текста. Сниппет представляет собой фрагмент из нескольких слов, который содержит нужное слово и по три слова справа и слева от него (если такие есть). (3 балла)

    а) Дополнительно: функция `kwiq` должна иметь keyword argument `num`, которое задает количество слов в сниппете вокруг искомого слова. По умолчанию `num` равен 3. (2 балла)

    б) Дополнительно: напишите функцию `to_table`, которая красиво оформляет результат в виде таблицы keyword in context (см. ниже). Эту функцию можно вызывать внутри `kwiq`, после нее и вообще как вам будет представляться разумным. (2 балла)

    читателям, что греческое    слово    анабазис означает военный
    "Домик" ― правильное        слово    в адрес квартиры,
    вставать. Это проклятое     слово    ― надо. Не хочешь
    реагирует на любое          слово    хирурга, боится простейших

2) К каждой функции, которую вы написали в коде, нужно обязательно добавить документацию (*docstring*). (1 балл)

3) Напишите `TestCase` для тестирования функции `kwiq`. (2 балла)